Gcode 71

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 


In [2]:
model = AbstractModel()
model.gen=Set()
model.t=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.RU=Param(model.gen)
model.RD=Param(model.gen)

model.demand=Param(model.t)

#Scalar
model.soco=Param(initialize=100)
model.socm=Param(initialize=300)
model.etac=Param(initialize=0.95)
model.etad=Param(initialize=0.90)


In [3]:
#constraint
def Pbounds(model,gen,t):
  return (model.Pmin[gen],model.Pmax[gen])

def Sbounds(model, t):
    return (0.2 * model.soco, model.socm)

def pcbounds(model,t):
    return (0,0.2*model.socm)

def pdbounds(model,t):
    return (0,0.2*model.socm)

model.p=Var(model.gen,model.t,bounds=Pbounds,within=Reals)
model.cost=Var(within=Reals)
model.soc=Var(model.t,bounds=Sbounds,within=Reals)
model.pd=Var(model.t,bounds=pdbounds,initialize=0,within=NonNegativeReals)
model.pc=Var(model.t,bounds=pcbounds,initialize=0,within=NonNegativeReals)


def rule1(model,t):
  return sum(model.p[gen,t] for gen in model.gen) + model.pd[t] >= model.demand[t] + model.pc[t]
model.c1 = Constraint(model.t,rule=rule1)

def rule2(model,gen,t):
    if t==1:
      return model.p[gen,t+1] - model.p[gen,t] <= model.RU
    else:
        return Constraint.Skip
model.c2 = Constraint(model.gen,model.t,rule=rule2)

def rule3(model,gen,t):
    if t==1:
      return model.p[gen,t-1]- model.p[gen,t] <= model.RD
    else:
        return Constraint.Skip
model.c3 = Constraint(model.gen,model.t,rule=rule3)

def rule6(model, t):
    if t == 't1':
        return model.soc[t] == model.soco + model.etac * model.pc[t] - model.pd[t] / model.etad
    else:
        previous_t = 't' + str(int(t[1:]) - 1)
        return model.soc[t] == model.soc[previous_t] + model.etac * model.pc[t] - model.pd[t] / model.etad
model.c6 = Constraint(model.t, rule=rule6)

# Add a constraint to ensure model.soc[t24] equals 100
def rule_final_soc(model):
    return model.soc['t24'] == 100
model.c_final_soc = Constraint(rule=rule_final_soc)


def ruleTC(model):
  return model.cost == sum((model.a[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.b[gen]*model.p[gen,t]+model.c[gen]) for gen in model.gen for t in model.t) 
model.c5 = Constraint(rule=ruleTC)



In [4]:
model.obj=Objective(expr=model.cost,sense=minimize)

In [5]:
instance= model.create_instance("code41.dat")

In [6]:
results = SolverFactory('ipopt').solve(instance).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 50
  Number of variables: 169
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.14.10\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.1202690601348877
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [7]:
instance.cost.pprint()

cost : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :  None : 645397.3395475473 :  None : False : False :  Reals


In [8]:
instance.p.pprint()

p : Size=96, Index=gen*t
    Key           : Lower : Value              : Upper : Fixed : Stale : Domain
     ('G1', 't1') :    28 :  180.8269177878211 :   200 : False : False :  Reals
    ('G1', 't10') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't11') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't12') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't13') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't14') :    28 : 200.00000199973405 :   200 : False : False :  Reals
    ('G1', 't15') :    28 : 200.00000199968082 :   200 : False : False :  Reals
    ('G1', 't16') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't17') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't18') :    28 : 200.00000199978015 :   200 : False : False :  Reals
    ('G1', 't19') :    28 : 200.00000199978015 :   200 : False : False :  Reals
     ('G1', 't2

In [9]:
instance.soc.pprint()

soc : Size=24, Index=t
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
     t1 :  20.0 : 143.62500039503718 : 300.0 : False : False :  Reals
    t10 :  20.0 : 243.55555792400145 : 300.0 : False : False :  Reals
    t11 :  20.0 : 203.66666883714166 : 300.0 : False : False :  Reals
    t12 :  20.0 : 150.44444655111405 : 300.0 : False : False :  Reals
    t13 :  20.0 : 103.88889086459785 : 300.0 : False : False :  Reals
    t14 :  20.0 : 103.88889086507076 : 300.0 : False : False :  Reals
    t15 :  20.0 : 103.88889086622709 : 300.0 : False : False :  Reals
    t16 :  20.0 :  95.11111257720515 : 300.0 : False : False :  Reals
    t17 :  20.0 :  93.00000088179404 : 300.0 : False : False :  Reals
    t18 :  20.0 : 38.666667495220615 : 300.0 : False : False :  Reals
    t19 :  20.0 : 22.111111506011046 : 300.0 : False : False :  Reals
     t2 :  20.0 : 168.25000098184472 : 300.0 : False : False :  Reals
    t20 :  20.0 : 19.999999801427247 : 300.0 : False : False :  Rea

In [10]:
instance.pd.pprint()

pd : Size=24, Index=t
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
     t1 :     0 :  -9.74610452406976e-09 :  60.0 : False : False : NonNegativeReals
    t10 :     0 :     21.900000310489737 :  60.0 : False : False : NonNegativeReals
    t11 :     0 :     35.900000169822924 :  60.0 : False : False : NonNegativeReals
    t12 :     0 :      47.90000004907401 :  60.0 : False : False : NonNegativeReals
    t13 :     0 :     41.900000109513705 :  60.0 : False : False : NonNegativeReals
    t14 :     0 :  -8.73167186939856e-09 :  60.0 : False : False : NonNegativeReals
    t15 :     0 :  -9.29358695737496e-09 :  60.0 : False : False : NonNegativeReals
    t16 :     0 :      7.900000451768884 :  60.0 : False : False : NonNegativeReals
    t17 :     0 :      1.900000517519132 :  60.0 : False : False : NonNegativeReals
    t18 :     0 :      48.90000003956523 :  60.0 : False : False : NonNegativeReals
    t19 :     0 :     14.900000381937753 :  60.0 : False : False

In [11]:
instance.pc.pprint()

pc : Size=24, Index=t
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
     t1 :     0 :      45.92105303600861 :  60.0 : False : False : NonNegativeReals
    t10 :     0 : -9.767083335845858e-09 :  60.0 : False : False : NonNegativeReals
    t11 :     0 :  -9.76708230372123e-09 :  60.0 : False : False : NonNegativeReals
    t12 :     0 : -9.767080865529642e-09 :  60.0 : False : False : NonNegativeReals
    t13 :     0 : -9.767081763922617e-09 :  60.0 : False : False : NonNegativeReals
    t14 :     0 : -9.714686880264568e-09 :  60.0 : False : False : NonNegativeReals
    t15 :     0 : -9.652507346428454e-09 :  60.0 : False : False : NonNegativeReals
    t16 :     0 : -9.767085928859296e-09 :  60.0 : False : False : NonNegativeReals
    t17 :     0 : -9.767097966150627e-09 :  60.0 : False : False : NonNegativeReals
    t18 :     0 :  -9.76708081791125e-09 :  60.0 : False : False : NonNegativeReals
    t19 :     0 : -9.767084418376622e-09 :  60.0 : False : False

In [12]:
instance.pprint()

2 Set Declarations
    gen : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'G1', 'G2', 'G3', 'G4'}
    t : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24'}

15 Param Declarations
    Pmax : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :   200
         G2 :   290
         G3 :   190
         G4 :   260
    Pmin : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    28
         G2 :    20
         G3 :    30
         G4 :    20
    RD : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    40
         G2 :    30
         G3 :    30
         G4 :    50